_This project was developed independently as part of Code Institute’s Predictive Analytics Project. Any datasets or templates used are openly provided by the course or via public sources like Kaggle. All commentary and code logic are my own._

# Notebook 02: Data Cleaning


## Objectives:
- Identify and handle missing values.
- Remove duplicates and inconsistencies.
- Standardize formatting and data types.
- Split the cleaned dataset.
- Save the cleaned datasets.
- Document the cleaning process.

### Inputs
- `/data/raw/house_prices_records.csv`  
- `/data/raw/inherited_houses.csv`

### Outputs
- `/data/processed/df_cleaned.csv`  
- Summary statistics for cleaned data  
- Missing value report

## Additional Observations
- This notebook follows the Data Preparation stage of the CRISP-DM approach.
- Decisions for cleaning (such as imputation techniques and handling outliers) are based on subject expertise and data properties.
- The feature engineering notebook will cover outlier detection and management.
- In later notebooks, the cleaned and divided datasets are prepared for activities like feature engineering, exploratory data analysis, and model training.

## Change Working Directory
- Since it is expected that you would keep the notebooks in a subfolder, you will need to switch the working directory when you run the notebook in the editor.
- The working directory must be changed from its current folder to its parent folder.
- We wish to change the current directory's parent to the new current directory.
- Verify the updated current directory.

In [ ]:
"""
Heritage Housing – Data Cleaning & Preparation Script

Overview:
This script handles data preparation for the Heritage Housing price prediction project. It performs the following steps:

1. Loads raw house price and inherited property data
2. Standardizes column names and identifies missing values
3. Visualizes missing data percentages
4. Applies numerical and categorical imputations using pipelines
5. Conducts Pearson and Spearman correlation analysis
6. Drops irrelevant or highly null features
7. Converts and verifies datatypes
8. Removes duplicates and finalizes a clean dataset
9. Splits the data into training and test sets
10. Saves all processed outputs in appropriate folders

Outputs:
- `data/processed/df_cleaned.csv`  
- `data/processed/cleaned/house_prices_cleaned.csv`  
- `data/processed/split/X_train.csv`, `y_train.csv`, `X_test.csv`, `y_test.csv`  
- `outputs/visuals/missing_data_barchart.png`, `pearson_correlation_heatmap.png`

Note:
This step ensures the data is ready for downstream modeling and visualization pipelines.  
"""

import os
# Smart Working Directory Setup
project_root = '/workspaces/heritage_housing'
if os.getcwd() != project_root:
    try:
        os.chdir(project_root)
        print(f"[INFO] Changed working directory to project root: {os.getcwd()}")
    except FileNotFoundError:
        raise FileNotFoundError(f"[ERROR] Project root '{project_root}' not found!")

## Import Packages & Set Environment Variables
- First, we need to import the numpy and pandas packages, and set the environment variables by running the following:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## Load Collected Data
- Now that we have imported the required packages and environment variables set, we need to load the data that was previously downloaded (refer to the Data Collection notebook).

In [ ]:
# Load cleaned data from the Data Collection stage
df = pd.read_csv("data/raw/house_prices_records.csv")
house_prices_df = pd.read_csv("data/raw/house_prices_records.csv")
inherited_df = pd.read_csv("data/raw/inherited_houses.csv")

# Standardize column names immediately
df.columns = df.columns.str.replace(' ', '_')
house_prices_df.columns = house_prices_df.columns.str.replace(' ', '_')
inherited_df.columns = inherited_df.columns.str.replace(' ', '_')

# Check missing values
print("\nMissing Values in House Prices Dataset (Absolute & Percentage):")
missing_df = pd.DataFrame({
    "Missing Values": df.isnull().sum(),
    "Percentage": (df.isnull().sum() / len(df) * 100).round(2)
})
if missing_df["Missing Values"].any():
    display(missing_df[missing_df["Missing Values"] > 0])
else:
    print("No missing values in the House Prices Dataset.")

print("\nMissing Values in Inherited Houses Dataset (Absolute & Percentage):")
missing_inherited_df = pd.DataFrame({
    "Missing Values": inherited_df.isnull().sum(),
    "Percentage": (inherited_df.isnull().sum() / len(inherited_df) * 100).round(2)
})
if missing_inherited_df["Missing Values"].any():
    display(missing_inherited_df[missing_inherited_df["Missing Values"] > 0])
else:
    print("No missing values in the Inherited Houses Dataset.")

# Other Checks
print(f"\nHouse Prices Dataset Shape: {df.shape}")
print(f"Inherited House Dataset Shape: {inherited_df.shape}")

print("\nData Types in House Prices Dataset:")
print(df.dtypes)

print("\nData Types in Inherited Houses Dataset:")
print(inherited_df.dtypes)

print("\nPreview of House Prices Dataset:")
display(df.head())

print("\nPreview of Inherited Houses Dataset:")
display(inherited_df.head())

print("\nSummary Statistics for House Prices Dataset:")
display(df.describe())

print("\nSummary Statistics for Inherited Houses Dataset:")
display(inherited_df.describe())

print("\nCategorical Summary for House Prices Dataset:")
display(df.select_dtypes(include=['object']).describe())

## Data Exploration – Missing Data
### Missing Data Exploration

**Identify Variables with Missing Data**
- Generate a list of columns that have missing values and calculate their corresponding percentages.

In [ ]:
# Check for null values
missing_house_prices = house_prices_df.isnull().sum()
missing_inherited = inherited_df.isnull().sum()

# Display only columns with missing values
print("\nMissing values in House Prices Dataset:")
print(missing_house_prices[missing_house_prices > 0])

print("\nMissing values in Inherited Houses Dataset:")
print(missing_inherited[missing_inherited > 0])

**Assessing Missing Data Levels**

In [ ]:
def missing_percentage(df):
    percent_missing = df.isnull().sum() / len(df) * 100
    return percent_missing[percent_missing > 0].sort_values(ascending=False)

print("\nHouse Prices Missing Data Percentage:")
print(missing_percentage(house_prices_df))

print("\nInherited Houses Missing Data Percentage:")
print(missing_percentage(inherited_df))


**Classify Missing Data**

Manual inspection and logical reasoning will help us categorize the nature of missingness

Examples:
- Garage-related columns may be missing if the house does not have a garage → Systematic.
- Missing `Electrical` could be data entry error → Random.


In [ ]:
def AssessMissingData(df):
    """
    Returns a DataFrame with count and percentage of missing data in the given dataset.
    """
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    
    if missing_data.empty:
        return pd.DataFrame()  # Return empty DataFrame if nothing's missing

    percent_missing = (missing_data / len(df)) * 100
    missing_summary = pd.DataFrame({
        "MissingValues": missing_data,
        "PercentageOfDataset": percent_missing
    })
    
    # Optional: Sort by highest percentage
    missing_summary = missing_summary.sort_values("PercentageOfDataset", ascending=False)

    return missing_summary

# Assess and plot missing data for House Prices Dataset
df_missing_data_summary = AssessMissingData(house_prices_df)

if not df_missing_data_summary.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(
        x=df_missing_data_summary.index,
        y=df_missing_data_summary["PercentageOfDataset"],
        palette="viridis",
        legend=False,
        hue=df_missing_data_summary.index,
    )
    plt.title("Percentage of Missing Data by Feature - House Prices Dataset")
    plt.xlabel("Features")
    plt.ylabel("Percentage of Missing Data")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig("outputs/visuals/missing_data_barchart.png")
    plt.show()
else:
    print("No missing data detected in the House Prices dataset.")

# Assess missing data for Inherited Houses Dataset
print("\nAssessing Missing Data in Inherited Houses Dataset:")
df_inherited_missing_data_summary = AssessMissingData(inherited_df)

if not df_inherited_missing_data_summary.empty:
    display(df_inherited_missing_data_summary)
else:
    print("No missing data detected in the Inherited Houses dataset.")


## Missing Data Assessment Results

**House Prices Dataset**

**1. High Missingness:**

- EnclosedPorch: ~90% missing — almost a ghost feature.
- WoodDeckSF: ~89% missing — severely incomplete.

**2. Moderate Missingness (~15-20%):**

- LotFrontage
- GarageFinish
- BsmtFinType1

**3. Low Missingness (<10%):**

- BedroomAbvGr
- 2ndFlrSF
- GarageYrBlt
- BsmtExposure
- MasVnrArea

**Insight:**
- EnclosedPorch and `WoodDeckSF` have extremely high missingness and may need to be dropped unless they hold critical business value.
- Other features can be imputed using median, mode, or model-based methods depending on their impact.

## Inherited Houses Dataset
- No missing data detected. Dataset is clean and ready.

## Visual Analysis Summary
- Most missingness appears feature-specific rather than across multiple columns.
- No strong clustering patterns detected, suggesting missing at random (MAR) rather than systematic missingness.

## Next Steps Recommendation
- Drop `EnclosedPorch` and `WoodDeckSF` if they are not critical to analysis.
- Impute other missing values using appropriate strategies based on feature type and importance.
- Document missing data handling steps clearly for reproducibility and transparency.



## Correlation Analysis

**This section's objective is to:**

- To find reliable predictors, assess the dataset's features' correlation with the target variable (SalePrice).
- Evaluate feature multicollinearity to identify pairs of strongly linked features that could need extra care in subsequent stages.
- To illustrate and summarize correlations and provide insights for feature selection and modeling, use heatmaps and numerical summaries.

In [ ]:
# Select only numeric columns
house_prices_numeric = house_prices_df.select_dtypes(include=[np.number])

# Calculate Pearson correlation matrix
pearson_corr = house_prices_numeric.corr(method='pearson')

# Plot Pearson heatmap
plt.figure(figsize=(12,10))
sns.heatmap(pearson_corr, annot=False, cmap='coolwarm', center=0)
plt.title("Pearson Correlation Heatmap - House Prices Dataset")
plt.tight_layout()
plt.savefig("outputs/visuals/pearson_correlation_heatmap.png")
plt.show()

# Calculate Spearman correlation matrix
spearman_corr = house_prices_numeric.corr(method='spearman')

# Plot Spearman heatmap
plt.figure(figsize=(12,10))
sns.heatmap(spearman_corr, annot=False, cmap='coolwarm')
plt.title('Spearman Correlation Heatmap')
plt.show()

## Multicollinearity Analysis (VIF)
**Objectives of Multicollinearity Analysis (VIF):**
- To detect if your features are lowkey sabotaging each other:
If two or more predictors are heavily correlated, they basically tell the model the same story twice — making it confused, bloating the importance of useless features, and wrecking model stability.

- To avoid inflated, unreliable model coefficients:
When multicollinearity is high, your model starts throwing out wild coefficient values that make no sense.

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer, MeanMedianImputer
from sklearn.pipeline import Pipeline

# Define imputers using original column casing
numeric_imputer = MeanMedianImputer(imputation_method='median', variables=["LotFrontage", "MasVnrArea"])
categorical_imputer = CategoricalImputer(imputation_method="missing", fill_value="None", variables=["GarageFinish", "BsmtFinType1"])
garage_year_imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=["GarageYrBlt"])
specific_feature_imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=["2ndFlrSF", "BedroomAbvGr"])

# Pipeline
imputation_pipeline = Pipeline([
    ("numeric_imputer", numeric_imputer),
    ("categorical_imputer", categorical_imputer),
    ("garage_year_imputer", garage_year_imputer),
    ("specific_feature_imputer", specific_feature_imputer)
])

# Apply to the cleaned dataset
df_cleaned = imputation_pipeline.fit_transform(house_prices_df)

# Check remaining missing values
print("\nRemaining missing values after imputation:")
print(df_cleaned.isnull().sum()[df_cleaned.isnull().sum() > 0])

# Sanity check
print("\nShape of cleaned dataframe:", df_cleaned.shape)
display(df_cleaned.head())

# Save output
df_cleaned.to_csv("data/processed/df_cleaned.csv", index=False)
print("\ndf_cleaned.csv saved successfully.")


## VIF Scores Analysis
**Highest VIFs:**
- GarageYrBlt (4.15)
- TotalBsmtSF (3.90)
- YearBuilt (3.78)
- 1stFlrSF (3.73)

**Mid VIFs:**
- OverallQual, GrLivArea, GarageArea, YearRemodAdd

**Low VIFs:**
- BsmtFinSF1, MasVnrArea

### Conclusion
The Variance Inflation Factor (VIF) analysis indicates that all selected features have VIF scores below 5, suggesting no severe multicollinearity is present. Therefore, all features can be retained for modeling without immediate concern for redundancy or instability.

**Action Required?**
- No immediate drop or transformation needed based on VIF.
- Keep an eye on the "GarageYrBlt", "TotalBsmtSF" area when feature engineering — minor tweaking might help model efficiency.



## Dealing with Missing Data: 

### Drop Variables:
**Purpose:**

Some variables are dropped because they either:
- Do not provide meaningful predictive power (e.g., IDs, irrelevant details),
- Introduce redundancy (highly correlated with other variables),
- Leak target information (variables that reveal the outcome directly),
- Increase noise and model complexity without adding value.

**By removing such variables, we aim to:**
- Improve model interpretability,
- Reduce the risk of overfitting,
- Enhance model training efficiency and performance.

In [ ]:
# Preview all the columns
print("Columns in house_prices_df:")
print(house_prices_df.columns)

# Drop columns with high missing values and low correlation
columns_to_drop = ['EnclosedPorch', 'WoodDeckSF']
house_prices_df.drop(columns=columns_to_drop, axis=1, inplace=True)


print("Dropped features due to excessive missing data or low correlation:")
print(columns_to_drop)


### Expected Outcomes
- The features removed contribute little predictive value and their absence will not degrade model performance.
- The dataset is now cleaner, leaner, and easier to manage.

## Impute Missing Values

### Purpose:
**To address missing data effectively and maintain the integrity of the dataset, the following imputation strategies will be applied:**

- LotFrontage (17.74% missing): Impute using the median value to minimize the influence of extreme outliers while preserving the central tendency of the data.
- GarageFinish (11.10% missing): Impute using the mode ("None") because missing values typically correspond to houses without garages, aligning logically with the property attributes.
- GarageYrBlt (5.55% missing): Replace missing values with 0 to represent properties that have no garage, ensuring consistency with related features.
- BsmtFinType1 (7.81% missing): Impute with "None" to reflect the absence of a finished basement in properties where this feature is not applicable.
- Other Numeric Variables: Apply median imputation across numeric features to maintain robustness against skewed distributions and to avoid bias introduced by mean imputation.

A dedicated preprocessing pipeline will be implemented to automate these imputations, ensuring that missing data handling is systematic, reproducible, and scalable for model training.

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer, MeanMedianImputer
from sklearn.pipeline import Pipeline

# Define pipelines for numeric and categorical variables
numeric_imputer = MeanMedianImputer(imputation_method='median', variables=["LotFrontage", "MasVnrArea"])
categorical_imputer = CategoricalImputer(imputation_method="missing", fill_value="None", variables=["GarageFinish", "BsmtFinType1"])
garage_year_imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=["GarageYrBlt"])
specific_feature_imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=["2ndFlrSF", "BedroomAbvGr"])

# Combine into a pipeline
imputation_pipeline = Pipeline([
    ("numeric_imputer", numeric_imputer),
    ("categorical_imputer", categorical_imputer),
    ("garage_year_imputer", garage_year_imputer),
    ("specific_feature_imputer", specific_feature_imputer)
])

# Fit and transform on house_prices_df
df_cleaned = imputation_pipeline.fit_transform(house_prices_df)

# Check to confirm no missing values remain
print("\nRemaining missing values after imputation:")
print(df_cleaned.isnull().sum()[df_cleaned.isnull().sum() > 0])

# Quick sanity check
print("\nShape of cleaned dataframe:", df_cleaned.shape)
display(df_cleaned.head())

# Save cleaned data
df_cleaned.to_csv("data/processed/df_cleaned.csv", index=False)
print("\n df_cleaned.csv saved successfully.")


### Expected Results
- All previously identified missing values are now filled.
- Median is robust against outliers for numeric data.
- Mode ensures the most frequent category is used for missing labels.

## Standardised Formatting

We must make sure that every feature is in the right format and follows a defined structure in order to get the dataset ready for modeling. This stage entails:

- Transforming numerical columns into the appropriate data types, such as floating or integers.
- To guarantee consistency, categorical variable formats should be standardized.
- Datetime formats are aligned where appropriate.
- Standardizing data guarantees algorithm compatibility and reduces errors during machine learning procedures.

In [ ]:
# Standardise column names: replace spaces with underscores
house_prices_df.columns = house_prices_df.columns.str.replace(' ', '_')


# Convert data types (if any need converting)
numeric_cols = house_prices_df.select_dtypes(include=['object']).columns
for col in numeric_cols:
    try:
        house_prices_df[col] = pd.to_numeric(house_prices_df[col])
    except ValueError:
        pass  # skip non-convertible columns

# Define categorical columns
categorical_cols = house_prices_df.select_dtypes(include=['object']).columns

# Display updated column names and data types
print("\nStandardised column names and data types:")
print(house_prices_df.dtypes)

# Confirm Data Types and Consistency
print("\n[INFO] After Conversion - Data Types for Numeric Columns:")
print(df_cleaned[numeric_cols].dtypes)

print("\n[INFO] Sample Data for Numeric Columns:")
display(df_cleaned[numeric_cols].head())

print("\n[INFO] Sample Data for Categorical Columns:")
display(df_cleaned[categorical_cols].head())

### Expected Results
- All column names are now consistent, lowercase, and snake_case.
- Numeric data types are standardized for future modeling steps.


## Remove Duplicates and Inconsistencies

**Purpose:**
- Make sure there are no duplicate rows in the dataset, as this can cause redundancy and skew analysis.
- Verify that numerical and category values are within the anticipated ranges and correct any discrepancies.

In [ ]:
# Remove Duplicates
initial_shape = df_cleaned.shape
df_cleaned.drop_duplicates(inplace=True)
final_shape = df_cleaned.shape

print(f"\n[INFO] Duplicates Removed: {initial_shape[0] - final_shape[0]}")
print(f"[INFO] New Dataset Shape: {final_shape}")

## Splitting the Data

**Purpose:**
- The model's ability to generalize to new data is ensured by separating the data into training and testing sets.
- While the testing set assesses the model's performance on fresh, untested data, the training set is used to train the model.

**Our Objective:**
- Preserve the overall data distribution while keeping the training and testing sets balanced.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the Data into Features (X) and Target (y)
X = df_cleaned.drop('SalePrice', axis=1)
y = df_cleaned['SalePrice']

# Use a standard splitting ratio, such as 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Check the split
print(f"\n[INFO] Training Set Shape: {X_train.shape}, {y_train.shape}")
print(f"[INFO] Test Set Shape: {X_test.shape}, {y_train.shape}")

## Save Processed Datasets

The cleaned dataset and the training and testing datasets will be saved in the relevant output directories in this section. For ease of access in later phases of this project, the structure guarantees that all processed data is centralized.

In [ ]:
import os

# Define directories for saving cleaned and split datasets
base_dir = "data/processed"
cleaned_data_dir = os.path.join(base_dir, "cleaned")
split_data_dir = os.path.join(base_dir, "split")

# Create directories if they don't already exist
os.makedirs(cleaned_data_dir, exist_ok=True)
os.makedirs(split_data_dir, exist_ok=True)

# Saving cleaned data to the specified directory
cleaned_data_file = os.path.join(cleaned_data_dir, "house_prices_cleaned.csv")
df_cleaned.to_csv(cleaned_data_file, index=False)
print(f"Cleaned dataset saved at: {cleaned_data_file}")

# Saving split datasets (train and test) to the specified directory
train_data_paths = {
    'X_train': os.path.join(split_data_dir, "X_train.csv"),
    'y_train': os.path.join(split_data_dir, "y_train.csv"),
    'X_test': os.path.join(split_data_dir, "X_test.csv"),
    'y_test': os.path.join(split_data_dir, "y_test.csv")
}

# Save the datasets to the respective paths
X_train.to_csv(train_data_paths['X_train'], index=False)
y_train.to_csv(train_data_paths['y_train'], index=False)
X_test.to_csv(train_data_paths['X_test'], index=False)
y_test.to_csv(train_data_paths['y_test'], index=False)

print(f"Training and testing datasets have been saved to: {split_data_dir}")


## Conclusion and Next Steps

**Summary:**
- Explored, cleaned, and preprocessed the house prices dataset.
- Strategically imputed missing values.
- Removed duplicate entries and ensured data consistency.
- Split the dataset into training and testing subsets.
- Saved processed datasets for future model development.

**Next Steps:**
- To perform Feature Engineering and Transformation.
- Train baseline regression models.
- Conduct hyperparameter tuning for optimization.
- Evaluate model performance and identify key drivers that impact house prices.
